In [1]:
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
gpu_options = tf.GPUOptions(allow_growth=True)
sess_config = tf.ConfigProto(gpu_options=gpu_options)
tf.keras.backend.set_session(tf.Session(config=sess_config))

In [2]:
import os
import glob
#import tqdm
import numpy as np
#import pandas as pd
import multiprocessing
import pydicom as dicom
import nibabel as nib
#from keras import utils as kutils
from matplotlib import pyplot as plt

def load_image(label_fpath, transpose=False):
    #encode_name = label_fpath[-39: -7]
    label_data = nib.load(label_fpath)
    label_array = label_data.get_fdata()
    if transpose:
        label_array = np.transpose(label_array, axes=(2, 1, 0))
    return  label_array#, encode_name

In [3]:
import pandas as pd
info = pd.read_csv('/data2/pancreas/Nifti_data/data_list.csv')

In [4]:
image_path = "/data2/pancreas/Nifti_data/image"
label_path = "/data2/pancreas/Nifti_data/label"

In [8]:
resec0_names = info.groupby("resection").get_group(0).case_id.unique()
resec1_names = info.groupby("resection").get_group(1).case_id.unique()

#combine resec0 and resec1 and shuffle
resec = np.concatenate([resec0_names,resec1_names], axis = 0)



In [19]:
np.random.shuffle(resec)

In [20]:
y = np.zeros((len(resec), 1))
for idx, name in enumerate(resec):
    y[idx] = info[info.case_id == name].resection



In [28]:
np.where(y[-30:] == 1)[0].shape
np.where(y[-50:-30] == 1)[0].shape

(9,)

In [26]:
info

,patient_id,case_id,exam_date,type,cut,200_list,diff_patient_list,add_date,resection,stage,size
0,26,NP1,20180101.0,healthy,fine,validation,True,20190210,NaN,NaN,NaN
1,236,AD1,20170928.0,healthy,fine,validation,True,20190210,NaN,NaN,NaN
2,216,AD10,20160920.0,healthy,fine,train,True,20190210,NaN,NaN,NaN
3,185,AD100,20140102.0,healthy,fine,train,True,20190210,NaN,NaN,NaN
4,164,AD101,20140204.0,healthy,fine,test,True,20190210,NaN,NaN,NaN
5,152,AD102,20131205.0,healthy,fine,test,True,20190210,NaN,NaN,NaN
6,205,AD104,20130817.0,healthy,fine,train,True,20190210,NaN,NaN,NaN
7,188,AD105,20140123.0,healthy,fine,train,True,20190210,NaN,NaN,NaN
8,178,AD108,20131021.0,healthy,fine,test,True,20190210,NaN,NaN,NaN
9,217,AD11,20160826.0,healthy,fine,train,True,20190210,NaN,NaN,NaN


In [25]:
#check the num. of image in terms of z-axis
img_shape = []
for name in resec:
    img = load_image(os.path.join(image_path, "IM_"+name+".nii.gz"))
    print(img.shape[2])
    img_shape.append(img.shape[2])
    del img



array(['PC115', 'PT8', 'PC555', 'PC278', 'PC336', 'PC79', 'PT42', 'PT4',
       'PC2', 'PC560', 'PT35', 'PC589', 'PC97', 'PC579', 'PC89', 'PC181',
       'PC41', 'PC96', 'PC110', 'PC301', 'PC39', 'PC147', 'PT51', 'PC70',
       'PC322', 'PC629', 'PC607', 'PC335', 'PC553', 'PC548', 'PC58',
       'PC575', 'PC537', 'PC350', 'PC545', 'PC552', 'PC534', 'PC259',
       'PC116', 'PC631', 'PC590', 'PC618', 'PC1', 'PC644', 'PC267',
       'PC27', 'PC550', 'PC651', 'PT27', 'PC581', 'PC605', 'PC650',
       'PC170', 'PC40', 'PC600', 'PC536', 'PC134', 'PC54', 'PT1', 'PC130',
       'PC306', 'PC103', 'PC563', 'PT39', 'PC513', 'PC538', 'PC613',
       'PC602', 'PC570', 'PC387', 'PC304', 'PC159', 'PT49', 'PC256',
       'PC630', 'PC285', 'PC246', 'PC151', 'PC627', 'PC152', 'PC55',
       'PC573', 'PC639', 'PT6', 'PC619', 'PC567', 'PC556', 'PC601',
       'PC104', 'PC273', 'PC91', 'PC63', 'PC603', 'PC294', 'PC612',
       'PT13', 'PT37', 'PC582', 'PT53', 'PC623', 'PC559', 'PC107', 'PC42',
       'PC6

In [ ]:
#100:400 200:360 
#60:5, 40:4, +-20 median   24 inouts 
def clip_img(img):# make image to be in shape of (300, 160, 200)
    img = img[100:400, 200:360, :]
    _med = int(img.shape[2]/2)
    tmp_img = np.zeros((300, 160, 200))
    if img.shape[2] < 200:
        a = img.shape[2]
        tmp_img[:, :, 100-_med : 100-_med+a] = img
    else:
        tmp_img = img[:, :, _med-100 : _med+100 ]
    img = tmp_img
    del tmp_img
    return img
    
def extract_cube(img, i):#Input((60, 40, 30, 1)) ; the i-th cube
    j = i%5
    k = i%4
    l = i%2
    #print(img[j*60:(j+1)*60, k*40:(1+k)*40, l*30:(l+1)*30].shape)
    return img[j*60:(j+1)*60, k*40:(1+k)*40, l*100:(l+1)*100]
    
    
import pandas as pd
#y_idx => for construct (500, 1) vector
map_df = pd.DataFrame(data={'resec_name': resec, 'target':y.reshape(182)})

In [ ]:
from keras.utils import to_categorical

class resec_Generator(kutils.Sequence):
    'Generates data for Keras'
    def __init__(self, mapping_df, batch_size, shuffle=True):
        'Initialization'
        self.mapping_df = mapping_df
        self.data_num   = mapping_df.shape[0]
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.data_num / self.batch_size))


    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        #print("enter0")
        batch_mapping_df = \
            self.mapping_df.iloc[index*self.batch_size: (index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(batch_mapping_df)
        return X, y


    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle:
            self.mapping_df = self.mapping_df.sample(frac=1).reset_index(drop=True)
            
    def __data_generation(self, batch_mapping_df):
        'Generates data containing batch_size samples' 
        # Initialization
        X = [np.zeros((self.batch_size, 60, 40, 100, 1)) for i in range(40)]
        y = np.zeros((self.batch_size))

        # Generate data
        cnt = 0
        
        for i, row in batch_mapping_df.iterrows():
            img = clip_img(load_image(os.path.join(image_path, "IM_"+row['resec_name']+".nii.gz")))
            #print("resec_name =", row['resec_name'])
            for j in range(40):
                #print("img shape =", img.shape)
                #print("j =", j)
                X[j][cnt, :, :, :, 0] = extract_cube(img, j)
            y[cnt] = row['target']

#             position = extract_position(row['ref_idx'], row['point_idx'])
#             tmp_cube = extract_cube(position, row['ref_idx'])
#             if (tmp_cube.shape == (19, 19, 19)):
#                 X[cnt, :, :, :, 0] = extract_cube(position, row['ref_idx'])
#             else:
#                 X[cnt, :, :, :, 0] = np.zeros((19, 19, 19))
#             y_idx = row['y_idx']
#             radius = ref[row['ref_idx']][row['point_idx'], 3]
#             y1[cnt, y_idx] = 1
#             y2[cnt, 0] = radius
            cnt += 1
        #print(X[5])
        return X, y

# for name in resec:
#     img = load_image(os.path.join(image_path, "IM_"+name+".nii.gz"))

In [ ]:
batch_size = 1
v_generator = resec_Generator(map_df[:-50], batch_size=batch_size)
validation_generator = resec_Generator(map_df[-50:-30], batch_size=batch_size)